Arrumando os arquivos

In [79]:
arquivos = ['ratings.csv', 'targets.csv']

for arquivo in arquivos:
    with open(arquivo, 'r') as file:
        conteudo = file.read()

    conteudo_modificado = conteudo.replace(':', ',')

    with open(arquivo, 'w') as file:
        file.write(conteudo_modificado)

Imports + leitura dos arquivos

In [71]:
import numpy as np
import pandas as pd

df_ratings = pd.read_csv("ratings.csv")
df_submissions = pd.read_csv("submission-rand.csv")
df_targets = pd.read_csv("targets.csv")

# Treino
Vamos fazer uma abordagem Item-Based para uma filtragem colaborativa personalizada. Para isso vamos treinar o modelo usando os dados históricos fornecidos no arquivo **ratings.csv**

## 1º Passo: Criar um dicionário de item-usuário e um de usuário-item

In [72]:
user_dict = {}
item_dict = {}

for _, row in df_ratings.iterrows():
    user = row['UserId']
    item = row['ItemId']
    rating = row['Rating']
    
    if user not in user_dict:
        user_dict[user] = {}  # Inicializa o dicionário para o usuário
    
    user_dict[user][item] = rating  # Armazena a avaliação do usuário para o item
    
    if item not in item_dict:
        item_dict[item] = {}  # Inicializa o dicionário para o item
    
    item_dict[item][user] = rating  # Armazena a avaliação do usuário para o item


# 2º Passo: Normalizar os dados

In [73]:
def mean_center_normalization(dict):
    normalizado = {}
    for usuario_item, avaliacoes in dict.items():
        media = sum(avaliacoes.values()) / len(avaliacoes)
        normalizado[usuario_item] = {
            item: nota - media for item, nota in avaliacoes.items()
        }
    return normalizado


user_dict_norm = mean_center_normalization(user_dict)
item_dict_norm = mean_center_normalization(item_dict)

## 3º Passo: Definir uma métrica de similaridade
Nesse caso escolheremos a **similaridade do cosseno**

Os vizinhos próximos serão todos aqueles que já avaliaram o mesmo item -> `common_users`

In [74]:
# Função para calcular similaridade entre dois itens com base nas avaliações
def cosine_similarity(item1_ratings, item2_ratings):
    common_users = set(item1_ratings.keys()).intersection(set(item2_ratings.keys()))
    
    if len(common_users) == 0:
        return 0  # Se não houver usuários em comum, a similaridade é 0
    
    ratings1 = np.array([item1_ratings[user] for user in common_users])
    ratings2 = np.array([item2_ratings[user] for user in common_users])
    
    dot_product = np.dot(ratings1, ratings2)
    norm1 = np.linalg.norm(ratings1)
    norm2 = np.linalg.norm(ratings2)
    
    if norm1 == 0 or norm2 == 0:
        return 0
    
    return dot_product / (norm1 * norm2)

## 4º Passo: Executar o nosso recomendador
Para isso vamos levar em consideração que caso não haja similaridade, o valor da recomendação será a média dos valores já avaliados para aquele item. 

In [75]:
def media(ratings):
    aux = sum(ratings.values()) / len(ratings)
    return aux

In [76]:
# Função para prever a avaliação de um usuário para um item
def predict_rating(user, item):
    if user not in user_dict_norm or item not in item_dict_norm:
        aux = media(item_dict.get(item, {}))
        return aux  # Cold Start User ou "Cold Start Item", retorna a média das avaliações

    # Guarda as avaliações do usuário "user"
    user_ratings = user_dict_norm[user]
    
    if item in user_ratings:
        return user_ratings[item]  # Se o usuário já avaliou o item, retorna a avaliação
    
    # Calcular similaridade entre o item desejado e os itens avaliados pelo usuário
    similarities = []
    ratings = []
    
    for rated_item in user_ratings:
        if rated_item in item_dict_norm:
            similarity = cosine_similarity(item_dict_norm[rated_item], item_dict_norm.get(item, {}))
            similarities.append(similarity)
            ratings.append(user_ratings[rated_item])
    
    # Prever a nota como a média ponderada pelas similaridades
    if len(similarities) == 0 or sum(similarities) == 0:
        aux = media(item_dict.get(item, {}))
        return aux  # Se não houver similaridade suficiente, retorna a média das avaliações
    
    return np.dot(similarities, ratings) / sum(similarities)

In [77]:
# Exemplo de previsão
user = '33ce7ee122'
item = '1dfcdde662'

predicted_rating = predict_rating(user, item)
print(f"Avaliação prevista para o usuário {user} no item {item}: {predicted_rating}")

Avaliação prevista para o usuário 33ce7ee122 no item 1dfcdde662: 4.2


In [78]:
print("UserId:ItemId,Rating")
for _, row in df_targets.iterrows():
     user = row["UserId"]
     item = row["ItemId"]
     predicted_rating = predict_rating(user, item)
     print(f"{user}:{item},{predicted_rating}")

UserId:ItemId,Rating
33ce7ee122:34cb28c370,4.5
eab9e065e5:34cb28c370,4.5
f785763291:34cb28c370,4.5
5f8185d75d:34cb28c370,4.5
0eeef87507:1dfcdde662,4.2
cc5199f2f7:7724ef1f69,5.0
82ae2469c6:bc3b9136bc,4.0
afced32639:89138b3fef,4.5
cea1eee489:89138b3fef,4.5
2bb8f34435:ee46c55ce9,3.5
355c1965c6:47fbb47c3e,5.0
ee96e33972:47fbb47c3e,5.0
ad9b954e7d:47fbb47c3e,5.0
08094b7c2e:d9c70d1ac6,5.0
6d7faa7b56:d9c70d1ac6,5.0
e89adb845b:972e7ee0b6,4.857142857142857
678dab68a5:972e7ee0b6,4.857142857142857
8952fbc06b:972e7ee0b6,4.857142857142857
d024341d3b:565cf650b0,4.666666666666667
a463ad7c2e:751afdc39b,4.636363636363637
16f1eee90f:751afdc39b,4.636363636363637
2ac929c26f:751afdc39b,4.636363636363637
18e8d8d43c:e9440489d3,4.6
87b2ec63e1:26d30a08c5,3.466666666666667
f35698f27c:26d30a08c5,3.466666666666667
5f7978e83a:5e83d4a0b7,4.2
c032d35dbb:26d30a08c5,3.466666666666667
8e5c639a87:eb98bae7af,4.375
c0a8373027:a13b02f7c3,4.589743589743589
6e1fd87e63:c1c4ae27c3,3.9411764705882355
891382c830:a13b02f7c3,4.5897